[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/unboxai/examples-gallery/blob/main/text-classification/transformers/transformers.ipynb)


# Text classification using HuggingFace Transformers

This notebook illustrates how transformers can be upladed to the Unbox platform.

## Importing the modules

In [1]:
from scipy.special import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

## Loading the pre-trained model and tokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

## Unbox part!

### Instantiating the client

In [4]:
import unboxapi

client = unboxapi.UnboxClient("YOUR_API_KEY_HERE")

### Creating a project

In [5]:
from unboxapi.tasks import TaskType

project = client.create_project(name="Transformer Demo Project",
                                task_type=TaskType.TextClassification,
                                description="Project to Demo Transformers with Unbox")

Created your project. Check out https://unbox.ai/projects!


### Uploading the model

First, it is important to create a `predict_proba` function, which is how Unbox interacts with your model

In [6]:
def predict_proba(model, texts, tokenizer):
    
    batch = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    
    with torch.no_grad():        
        outputs = model(**batch)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        probs = predictions.detach().numpy().tolist()
        
        return probs

Let's test the `predict_proba` function to make sure the input-output format is consistent with what Unbox expects:

In [7]:
predict_proba(model, ["good morning!", "I've had a horrible day."], tokenizer)

[[0.00017212914826814085, 0.9998278617858887],
 [0.999519944190979, 0.0004800486785825342]]

Now, we can upload the model:

In [8]:
from unboxapi.models import ModelType

model = project.add_model(
    function=predict_proba, 
    model=model,
    model_type=ModelType.transformers,
    class_names=['negative', 'positive'],
    name='transformers.sentiment_analyzer',
    commit_message='this is my transformers sentiment model',
    tokenizer=tokenizer,
    requirements_txt_file='./requirements.txt'
)

Bundling model and artifacts...
Uploading model to Unbox! Check out https://unbox.ai/models to have a look!
